In [ ]:
from rusty_tags import *
from rusty_tags.datastar import Signals
from rusty_tags.utils import create_template, show

hdrs = (
    Script(src="https://cdn.jsdelivr.net/gh/starfederation/datastar@main/bundles/datastar.js", type="module"),
)
htmlkws = dict(cls="bg-background text-foreground font-sans antialiased")
bodykws = dict(cls="h-screen p-16 bg-white text-foreground font-sans antialiased")
page = create_template(hdrs=hdrs, htmlkw=htmlkws, bodykw=bodykws)

@page(title="D* Demo")
def index():
    return Div(
        H1(text="$message", cls="text-4xl font-bold"),
        signals=Signals(message="Hello, from datastar signals!"),
    )

indexPage = index()
print(indexPage)
show(indexPage)

<!doctype html><html class="bg-background text-foreground font-sans antialiased"><head><title>D* Demo</title><script type="module" src="https://cdn.jsdelivr.net/gh/starfederation/datastar@main/bundles/datastar.js"></script><script src="https://cdn.jsdelivr.net/gh/starfederation/datastar@main/bundles/datastar.js" type="module"></script></head><body class="h-screen p-16 bg-white text-foreground font-sans antialiased"><div data-signals="{'message':'Hello, from datastar signals!'}"><h1 class="text-4xl font-bold" data-text="$message"></h1></div></body></html>


In [10]:
from pydantic import BaseModel, Field
from typing import Optional

class User(BaseModel):
    id: int = Field(description="The unique identifier for the user")
    name: str = Field(description="The name of the user")
    email: Optional[str] = Field(description="The email address of the user")
    age: Optional[int] = 100

for field, field_info in User.model_fields.items():
    print(f"{field.capitalize()}: {field_info.description} - Default: {field_info.default}")



Id: The unique identifier for the user - Default: PydanticUndefined
Name: The name of the user - Default: PydanticUndefined
Email: The email address of the user - Default: PydanticUndefined
Age: None - Default: 100


In [7]:
def display(component):
    return show(template(component))


In [8]:
def render_view(instance: BaseModel):
    components = []
    for field_name, field_info in instance.__class__.model_fields.items():
        value = getattr(instance, field_name)
        components.append(
            Div(
                Label(f"{field_info.title or field_name.capitalize()} :", cls="uk-label text-md"),
                Span(str(value), title=field_info.description or "", cls="text-md"),
                cls="border rounded-md mb-2 p-1"
            )
        )
    return Div(*components)

user = User(id=1, name="John Doe", email="john.doe@example.com")
display(render_view(user))

NameError: name 'template' is not defined

In [ ]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Any

class SignalDescriptor:
    """Return `$Model.field` on the class, real value on an instance."""

    def __init__(self, field_name: str) -> None:
        self.field_name = field_name

    def __get__(self, instance, owner):
        #  class access  →  owner is the model class, instance is None
        if instance is None:
            ns = owner._namespace
            use_ns = owner._use_namespace
            return f"${ns}.{self.field_name}" if use_ns else f"${self.field_name}"

        #  instance access  →  behave like a normal attribute
        return instance.__dict__[self.field_name]
    
class Signals(BaseModel):
    model_config = ConfigDict(extra="allow")
    _namespace = "signals"
    _use_namespace = False

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, f"_{k}", k)
            setattr(self, f"_S{k}", f"${k}")

    @classmethod
    def __pydantic_init_subclass__(cls, **kwargs: Any) -> None:
        super().__pydantic_init_subclass__(**kwargs)
        # Create signal descriptors for all model fields
        for field_name in cls.model_fields:
            setattr(cls, f"S{field_name}", SignalDescriptor(field_name))
        for field_name in cls.model_computed_fields:
            setattr(cls, f"S{field_name}", SignalDescriptor(field_name))

sigs = Signals(name="John Doe", age=100)
print(sigs._Sname, sigs._age)


$name age


In [ ]:
class User(Signals):
    name: str
    age: int

user = User(name="John Doe", age=100)
print(user._Sname, user._name, user.name)
User.Sname

$name name John Doe


"$default='signals'.name"